# Unscented Kalman Filter in pykal

This notebook demonstrates the Unscented Kalman Filter (UKF) for nonlinear state estimation, based on Julier & Uhlmann's seminal work [1].

## Introduction

The UKF addresses a key limitation of the Extended Kalman Filter (EKF): **linearization errors**. While the EKF linearizes nonlinear functions using first-order Taylor expansion (Jacobians), the UKF uses the **unscented transformation** to capture the true mean and covariance through nonlinear mappings up to the third order.

### Key Advantages of UKF:

1. **No Jacobians required**: Eliminates error-prone analytic derivatives
2. **Better accuracy**: Captures higher-order moments of nonlinear transformations
3. **Similar computational cost**: O(n³) like EKF, where n is state dimension
4. **Easier to implement**: Just provide f() and h() functions

### The Unscented Transformation:

The UKF uses **sigma points** to represent the state distribution:
- Generate 2n+1 weighted points from mean and covariance
- Propagate each point through the nonlinear function
- Reconstruct the mean and covariance from transformed points

This captures the nonlinear effects without linearization!

## Example Problem: Range-Bearing Tracking

We'll track an object moving in 2D using **range and bearing measurements** - a classic nonlinear problem where UKF excels.

**State**: $x = [p_x, p_y, v_x, v_y]^T$ (position and velocity)

**Dynamics** (constant velocity):
$$
\begin{bmatrix} p_x \\ p_y \\ v_x \\ v_y \end{bmatrix}_{k+1} = 
\begin{bmatrix} 1 & 0 & dt & 0 \\ 0 & 1 & 0 & dt \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 1 \end{bmatrix}
\begin{bmatrix} p_x \\ p_y \\ v_x \\ v_y \end{bmatrix}_k + w_k
$$

**Measurements** (range and bearing - **highly nonlinear**):
$$
y = \begin{bmatrix} r \\ \theta \end{bmatrix} = 
\begin{bmatrix} \sqrt{p_x^2 + p_y^2} \\ \arctan2(p_y, p_x) \end{bmatrix} + v_k
$$

## References

[1] Julier, S. J., & Uhlmann, J. K. (2004). Unscented filtering and nonlinear estimation. *Proceedings of the IEEE*, 92(3), 401-422.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pykal import DynamicalSystem
from pykal.algorithm_library.estimators import ukf

## System Model

Define the nonlinear dynamics and measurement functions.

In [ ]:
# Simulation parameters
dt = 0.1  # 10 Hz
T = 20.0  # 20 second simulation
N = int(T / dt)
time = np.arange(N) * dt

# State dimension: [px, py, vx, vy]
n = 4

# Dynamics function (constant velocity model)
def f(x, dt=dt):
    """
    State evolution: constant velocity motion.
    x = [px, py, vx, vy]
    """
    F = np.array([
        [1, 0, dt, 0],
        [0, 1, 0, dt],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])
    return F @ x

# Measurement function (range and bearing - NONLINEAR)
def h(x):
    """
    Nonlinear measurement: range and bearing.
    y = [range, bearing]
    """
    px, py, _, _ = x
    r = np.sqrt(px**2 + py**2)
    theta = np.arctan2(py, px)
    return np.array([r, theta])

# Process noise covariance
q_vel = 0.1  # Velocity uncertainty
Q = np.diag([0.01, 0.01, q_vel, q_vel])  # Higher uncertainty in velocity

# Measurement noise covariance
sigma_r = 0.3  # 30 cm range noise
sigma_theta = 0.05  # ~3 degree bearing noise
R = np.diag([sigma_r**2, sigma_theta**2])

print("✓ Nonlinear system model configured")
print(f"  State: [px, py, vx, vy] (4D)")
print(f"  Measurement: [range, bearing] (2D - nonlinear!)")
print(f"  Process noise: Q = diag([0.01, 0.01, {q_vel}, {q_vel}])")
print(f"  Measurement noise: R = diag([{sigma_r**2:.3f}, {sigma_theta**2:.4f}])")

## Generate True Trajectory and Measurements

Create a circular trajectory to test the UKF.

In [ ]:
# Generate true trajectory (circular motion)
radius = 10.0  # 10 meter radius
omega = 0.3    # Angular velocity

true_states = np.zeros((N, 4))
measurements = np.zeros((N, 2))

for k in range(N):
    t = time[k]
    # True state (circular trajectory)
    true_states[k, 0] = radius * np.cos(omega * t)  # px
    true_states[k, 1] = radius * np.sin(omega * t)  # py
    true_states[k, 2] = -radius * omega * np.sin(omega * t)  # vx
    true_states[k, 3] = radius * omega * np.cos(omega * t)   # vy
    
    # Generate noisy measurement
    true_meas = h(true_states[k])
    measurements[k] = true_meas + np.random.multivariate_normal([0, 0], R)

print("✓ Generated circular trajectory")
print(f"  Radius: {radius} m")
print(f"  Period: {2*np.pi/omega:.1f} seconds")
print(f"  Tangential velocity: {radius*omega:.2f} m/s")

## Run UKF Estimation

Apply the Unscented Kalman Filter to estimate the state from noisy measurements.

In [ ]:
# Initial state estimate (with error)
x0 = np.array([12.0, 2.0, 0.0, 0.0])  # Start with some error
P0 = np.diag([5.0, 5.0, 2.0, 2.0])   # Large initial uncertainty

# UKF parameters
params = {
    'xhat_P': (x0, P0),
    'yk': measurements[0],
    'f': f,
    'f_params': {'dt': dt},
    'h': h,
    'h_params': {},
    'Qk': Q,
    'Rk': R,
    'alpha': 1e-3,  # UKF spread parameter
    'beta': 2.0,    # Optimal for Gaussian distributions
    'kappa': 0.0    # Secondary scaling
}

# Create UKF dynamical system
ukf_filter = DynamicalSystem(
    f=ukf.UKF.f,
    h=ukf.UKF.h,
    state_name='xhat_P'
)

# Storage for estimates
estimates = np.zeros((N, 4))
covariances = np.zeros((N, 4, 4))

# Run UKF
print("Running UKF...")
for k in range(N):
    # Update measurement
    params['yk'] = measurements[k]
    
    # UKF step
    x_est = ukf_filter.step(params=params)
    
    # Store results
    estimates[k] = x_est
    covariances[k] = params['xhat_P'][1]

print("✓ UKF estimation complete!")
print(f"  Processed {N} measurements")

## Visualize Results

Compare true trajectory, measurements, and UKF estimates.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# Plot 1: 2D trajectory
ax = axes[0, 0]
ax.plot(true_states[:, 0], true_states[:, 1], 'g-', linewidth=2, label='True trajectory', alpha=0.7)
ax.plot(estimates[:, 0], estimates[:, 1], 'b--', linewidth=2, label='UKF estimate')
ax.scatter(estimates[0, 0], estimates[0, 1], c='red', s=100, marker='o', label='Start', zorder=5)
ax.set_xlabel('X Position (m)', fontsize=11)
ax.set_ylabel('Y Position (m)', fontsize=11)
ax.set_title('2D Trajectory Tracking', fontsize=13, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
ax.axis('equal')

# Plot 2: Position errors with uncertainty bounds
ax = axes[0, 1]
pos_error = np.sqrt(np.sum((estimates[:, :2] - true_states[:, :2])**2, axis=1))
pos_std = np.sqrt(covariances[:, 0, 0] + covariances[:, 1, 1])
ax.plot(time, pos_error, 'b-', linewidth=1.5, label='Position error')
ax.fill_between(time, 0, 3*pos_std, alpha=0.3, label='3σ uncertainty')
ax.set_xlabel('Time (s)', fontsize=11)
ax.set_ylabel('Position Error (m)', fontsize=11)
ax.set_title('Position Estimation Error', fontsize=13, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# Plot 3: Velocity errors
ax = axes[1, 0]
vel_error = np.sqrt(np.sum((estimates[:, 2:] - true_states[:, 2:])**2, axis=1))
vel_std = np.sqrt(covariances[:, 2, 2] + covariances[:, 3, 3])
ax.plot(time, vel_error, 'r-', linewidth=1.5, label='Velocity error')
ax.fill_between(time, 0, 3*vel_std, alpha=0.3, color='red', label='3σ uncertainty')
ax.set_xlabel('Time (s)', fontsize=11)
ax.set_ylabel('Velocity Error (m/s)', fontsize=11)
ax.set_title('Velocity Estimation Error', fontsize=13, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# Plot 4: Measurement residuals
ax = axes[1, 1]
# Compute predicted measurements
pred_meas = np.array([h(estimates[k]) for k in range(N)])
residual_r = measurements[:, 0] - pred_meas[:, 0]
residual_theta = measurements[:, 1] - pred_meas[:, 1]
ax.plot(time, residual_r, 'b-', linewidth=1, label='Range residual', alpha=0.7)
ax.plot(time, residual_theta, 'r-', linewidth=1, label='Bearing residual', alpha=0.7)
ax.axhline(0, color='k', linestyle='--', linewidth=0.5)
ax.set_xlabel('Time (s)', fontsize=11)
ax.set_ylabel('Measurement Residual', fontsize=11)
ax.set_title('Innovation Sequence', fontsize=13, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print performance metrics
print("\n" + "="*50)
print("UKF Performance Metrics")
print("="*50)
rmse_pos = np.sqrt(np.mean(pos_error[10:]**2))  # Skip initial transient
rmse_vel = np.sqrt(np.mean(vel_error[10:]**2))
print(f"Position RMSE (steady-state): {rmse_pos:.3f} m")
print(f"Velocity RMSE (steady-state): {rmse_vel:.3f} m/s")
print(f"Final position error: {pos_error[-1]:.3f} m")
print(f"Final velocity error: {vel_error[-1]:.3f} m/s")
print("="*50)

## UKF vs EKF Comparison

The UKF excels in this problem because:

1. **No Jacobian errors**: The measurement function $h(x) = [\sqrt{p_x^2 + p_y^2}, \arctan2(p_y, p_x)]$ has complex derivatives that can introduce linearization errors in EKF

2. **Better uncertainty propagation**: The unscented transformation captures the true shape of the transformed distribution, while EKF assumes Gaussian after linearization

3. **Same computational cost**: Both are O(n³), but UKF is often easier to implement (no Jacobians to derive)

### When to Use UKF:
- **High nonlinearity**: When f() or h() are strongly nonlinear
- **Large uncertainties**: When covariances are large and linearization breaks down
- **Rapid prototyping**: When you want to avoid deriving Jacobians

### When EKF Might Be Better:
- **Nearly linear systems**: When Jacobians are accurate
- **Very high dimensions**: UKF requires 2n+1 sigma points (can be costly for n >> 10)
- **Legacy code**: When EKF is already implemented and working well

## Tuning UKF Parameters

The UKF has three tuning parameters:

### Alpha (α)
- **Range**: 10⁻⁴ to 1
- **Effect**: Controls spread of sigma points around mean
- **Typical**: 10⁻³ (small spread, conservative)
- **Use larger α**: When process is highly nonlinear

### Beta (β)
- **Range**: β ≥ 0
- **Effect**: Incorporates prior knowledge of distribution
- **Typical**: 2 (optimal for Gaussian)
- **Use β = 0**: For uniform distributions

### Kappa (κ)
- **Range**: Any real number
- **Effect**: Secondary scaling parameter
- **Typical**: 0 or (3 - n)
- **Effect is small**: Usually keep at 0

In [ ]:
# Example: Testing different alpha values
print("UKF Parameter Tuning Guidelines:")
print("\nAlpha (spread):")
print("  α = 1e-4: Very tight spread (conservative, small nonlinearity)")
print("  α = 1e-3: Default (good for most applications)")
print("  α = 1e-1: Larger spread (for highly nonlinear systems)")
print("  α = 1.0:  Maximum spread (aggressive, may cause instability)")
print("\nBeta (distribution):")
print("  β = 2.0:  Optimal for Gaussian distributions")
print("  β = 0.0:  Use for uniform distributions")
print("\nKappa (secondary scaling):")
print("  κ = 0:    Default (simplest choice)")
print("  κ = 3-n:  Alternative (n = state dimension)")

## Summary

This notebook demonstrated:

1. ✅ **UKF implementation** in pykal for nonlinear state estimation
2. ✅ **Unscented transformation** to propagate uncertainty without Jacobians
3. ✅ **Range-bearing tracking** as a realistic nonlinear problem
4. ✅ **Performance analysis** showing estimation errors and uncertainty bounds
5. ✅ **Parameter tuning** guidelines for α, β, κ

**Key Takeaways:**
- UKF handles strong nonlinearities better than EKF
- No Jacobians required (easier implementation)
- Same O(n³) computational cost as EKF
- Sigma point approach captures higher-order moments

**Next Steps:**
- Deploy UKF on TurtleBot for nonlinear motion (`ukf_turtlebot.ipynb`)
- Use UKF for Crazyflie attitude estimation (`ukf_crazyflie.ipynb`)
- Compare with EKF on same problem
- Try different nonlinear systems (pendulum, spacecraft, etc.)

**References:**

[1] Julier, S. J., & Uhlmann, J. K. (2004). Unscented filtering and nonlinear estimation. *Proceedings of the IEEE*, 92(3), 401-422.